In [ ]:
%pip install chess
%pip install genanki
%pip install graphviz

## Attempt #2: Use graphs

In [1]:
import graphviz
import chess
import chess.svg
import hashlib
import re
import itertools
import math
import genanki
from os import listdir
from os.path import isfile, join

In [2]:
chess_model = genanki.Model(
    1146161877, # randomly chosen, must be unique
    "Chess Opening Model",
    fields = [
        {'name': 'position_fen'},
        {'name': 'question_move_text'}, # The moves that have been played so far. Might be multiline in case of transpositions.
        {'name': 'question_position'}, # The image where black hasn't moved yet
        {'name': 'question_move_comment'}, # Comments like "White develops the bishop"
        {'name': 'answer_move_text'}, # The move in textual representation
        {'name': 'answer_position'}, # A visual representation of the move on the answer card
        {'name': 'answer_move_comment'} # Comments like "Black responds by pinning the Knight."
    ],
    templates=[
        {
            'name': 'Card 1',
            'qfmt': '{{question_position}} <p>{{question_move_text}}</p>',
            'afmt': '{{answer_position}} <p>{{answer_move_text}}</p> <p><em>{{question_move_comment}}</em></p> <p><em>{{answer_move_comment}}</em></p>'
        }
    ],
    css="img {max-width: 500px;}"
)

In [12]:
lines = ""

from os import listdir
from os.path import isfile, join

for f in listdir("repertoire/Caro Kann Defense/"):
    with open("repertoire/Caro Kann Defense/" + f) as f:
        lines += f.read() + "\n"


In [13]:
chess_deck = genanki.Deck(
    2059400110,
    'Caro Kann Defense'
)

In [14]:
def md5(input: str) -> str:
    return hashlib.md5(input.encode("utf-8")).hexdigest()

In [15]:
def parse_move(move: str) -> tuple[str, str | None, str, str | None]:
    match = re.match(r"^([KNORQB\-a-hx+!0-8]+)(?: \((.+)\))? ([KNORQB\-a-hx+!0-8]+)(?: \((.+)\))?$", move)
    
    if not match:
        raise RuntimeError("Invalid move format! " + move)
    
    return match.group(1), match.group(2), match.group(3), match.group(4)

class EdgeData:
    def __init__(self, move: chess.Move, move_string: str, comments : list[str] | None = None):
        self.move = move
        self.move_string = move_string
        self.comments = comments or []

def fix_fen(fen: str) -> str:
    """
    This function is needed to set the turn numbers to 1, s.t. equal board positions are treated as equal (but keeping the information whose turn it is).
    """

    fen_parts = fen.split(" ")
    return " ".join(fen_parts[0:4]) + " 0 1" 

def build_graph():
    nodes = { fix_fen(chess.STARTING_FEN) }

    adjacency_list : dict[str, dict[str, EdgeData]] = {}

    board = chess.Board()

    for line in lines.split("\n"):
        line = line.strip()
        if line:
            temp = line.split(". ", maxsplit=1)
            move_number = int(temp[0])
            white_move_san, white_comment, black_move_san, black_comment = parse_move(temp[1])

            while move_number < board.fullmove_number:
                board.pop()
                board.pop()

            try:
                parent_position = fix_fen(board.fen())

                board.push_san(white_move_san)
                white_position = fix_fen(board.fen())
                white_move = board.peek()
                nodes.add(white_position)

                board.push_san(black_move_san)
                black_position = fix_fen(board.fen())
                black_move = board.peek()
                nodes.add(black_position)
            except:
                print("Error with line " + line)
                raise

            if parent_position not in adjacency_list: # The node is new and doesn't have any edges yet
                adjacency_list[parent_position] = {}

            if white_position not in adjacency_list: # The node is new and doesn't have any edges yet
                adjacency_list[white_position] = {}
           
            if black_position not in adjacency_list: # The node is new and doesn't have any edges yet
                adjacency_list[black_position] = {}

            if white_position not in adjacency_list[parent_position]: # The edge is new
                adjacency_list[parent_position][white_position] = EdgeData(white_move, white_move_san)

            if black_position not in adjacency_list[white_position]: # The edge is new
                adjacency_list[white_position][black_position] = EdgeData(black_move, black_move_san)

            if white_comment != None:
                adjacency_list[parent_position][white_position].comments.append(white_comment.strip())
                
            if black_comment != None:
                adjacency_list[white_position][black_position].comments.append(black_comment.strip())

    # handle transpositions
    for node in nodes:
        board = chess.Board(fen=node)

        for move in board.generate_legal_moves():
            board.push_uci(move.uci())

            position = fix_fen(board.fen())
            
            if position in nodes:
                if not position in adjacency_list[node]:
                    adjacency_list[node][position] = EdgeData(move, move.uci(), ["Transposed."])
            board.pop()

    return nodes, adjacency_list

In [16]:
def get_parent_nodes(adjacency_list, node):
    parent_nodes = []
    for key, edges in adjacency_list.items():
        for dest_node, edge_list in edges.items():
            if dest_node == node:
                parent_nodes.append(key)
    return parent_nodes

def build_visualization():
    dot = graphviz.Digraph("Caro Kann Defense")

    nodes, adjacency_list = build_graph()

    for position in nodes:
        board = chess.Board(fen=position)

        parent_nodes = get_parent_nodes(adjacency_list, position)

        shape = "box" if board.turn == chess.WHITE else "circle"
        style = "filled" if len(adjacency_list[position]) > 1 or len(parent_nodes) > 1 else ""
        color = "blue" if len(parent_nodes) > 1 else ("green" if len(adjacency_list[position]) > 1 else "")


        dot.node(position, label="", shape=shape, style=style, color=color)
        
        for target, edge_data in adjacency_list[position].items():
            dot.edge(position, target, edge_data.move_string)

    return dot

build_visualization()

ValueError: expected 'w' or 'b' for turn part of fen: 'r3kb1r/pp3ppp/4qn2/3p4/8/2P1BQ1P/PP3PP1/RN2K2RbKQkq- 0 1'

In [ ]:
def find_all_paths(graph: dict[str, dict[str, EdgeData]], start: str, end: str):
    paths : list[list[str]] = []
    visited = set()

    def depth_first_search(current_node, path):
        visited.add(current_node)

        if current_node == end:
            paths.append(path)
            return

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                depth_first_search(neighbor, path + [neighbor])
        
        visited.remove(current_node)
    
    depth_first_search(start, [start])
    return paths

In [ ]:
def path_to_line(adjacency_list, path: list[str]) -> list[str]:
    ret = []
    
    for values in itertools.pairwise(path):
        ret.append(adjacency_list[values[0]][values[1]].move_string)

    return ret

def to_move_string(moves: list[str]) -> str:
    if len(moves) == 0:
        return ""
    
    def break_into_pairs(lst):
        pairs = []
        if len(lst) % 2 != 0:
            lst.append(None)  # Add None to handle odd-length lists
        for i in range(0, len(lst), 2):
            pairs.append((lst[i], lst[i+1]))
        return pairs
    
    full_moves = break_into_pairs(moves)

    ret = ""
    
    i = 0
    for index, value in enumerate(full_moves[:-1]):
        ret += f"{index + 1}. {value[0]} {value[1]} "
        i = index

    white_move, black_move = full_moves[-1]
    if black_move is None:
        ret += f"{i + 1}. <strong>{white_move}</strong> "
    else:
        ret += f"{i + 1}. {white_move} <strong>{black_move}</strong> "

    return ret

In [ ]:
class ChessPositionNote(genanki.Note):
  @property
  def guid(self):
    return genanki.guid_for(self.fields[0])

In [ ]:
def write_cards():
    nodes, adjacency_list = build_graph()

    colors = {
        #'square dark lastmove': '#3c78ffd9',
        #'square light lastmove': '#3c78ffd9',
    }

    for position in nodes:
        board = chess.Board(fen=position)

        if board.turn == chess.WHITE:
            continue

        paths = find_all_paths(adjacency_list, fix_fen(chess.STARTING_FEN), position)
        
        last_move = None
        arrows = []

        question_lines = []
        question_comments = []
        question_image_path = md5(position)

        if len(paths) == 1:
            # In this case we can draw white's last move
            last_position = paths[0][-2]
            last_move = adjacency_list[last_position][position].move

            question_lines = [path_to_line(adjacency_list, paths[0])]
            question_comments = adjacency_list[last_position][position].comments
            question_image = chess.svg.board(board, orientation = chess.BLACK, lastmove=last_move, colors=colors)

        else:
            # Since there is not a single last move, we draw all possible last moves instead
            for path in paths:
                last_position = path[-2]

                move = adjacency_list[last_position][position].move
                arrows.append((move.from_square, move.to_square)) 

                question_lines.append(path_to_line(adjacency_list, path))
                question_comments.extend(adjacency_list[last_position][position].comments)
            question_image = chess.svg.board(board, orientation = chess.BLACK, arrows=arrows, colors=colors)

        answer_comments = []
        answer_lines = []

        continuations = adjacency_list[position]
        for c, edge_data in continuations.items():
            answer_lines.extend([line + [edge_data.move_string] for line in question_lines])
            answer_comments.extend(edge_data.comments)

        if len(continuations) == 1:
            next_move = next(iter(continuations.values())).move
            next_position = next(iter(continuations.keys()))
            board.push(next_move)
            answer_image = chess.svg.board(board, orientation = chess.BLACK, lastmove=next_move, colors=colors)
            answer_image_path = md5(next_position)
        else:
            arrows = [(c.move.from_square, c.move.to_square) for c in continuations.values()]
            answer_image = chess.svg.board(board, orientation = chess.BLACK, arrows=arrows, colors=colors)
            answer_image_path = question_image_path + "_a"

        with open("images/" + question_image_path + ".svg", "w") as f:
            f.write(question_image)

        with open("images/" + answer_image_path + ".svg", "w") as f:
            f.write(answer_image)

        if len(question_comments) > 1:
            question_comment_string = "<ol><li>" + "</li><li>".join(question_comments) + "</li></ol>"
        elif len(question_comments) == 1:
            question_comment_string = question_comments[0]
        else:
            question_comment_string = ""

        if len(question_lines) > 1:
            question_move_string = "<ol><li>" + "</li><li>".join(map(to_move_string, question_lines)) + "</li></ol>"
        elif len(question_lines) == 1:
            question_move_string = to_move_string(question_lines[0])
        else:
            question_move_string = ""

        if len(answer_lines) > 1:
            answer_move_string = "<ol><li>" + "</li><li>".join(map(to_move_string, answer_lines)) + "</li></ol>"
        elif len(answer_lines) == 1:
            answer_move_string = to_move_string(answer_lines[0])
        else:
            answer_move_string = ""


        if len(answer_comments) > 1:
            answer_comment_string = "<ol><li>" + "</li><li>".join(answer_comments) + "</li></ol>"
        elif len(answer_comments) == 1:
            answer_comment_string = answer_comments[0]
        else:
            answer_comment_string = ""

        note = ChessPositionNote(
            model=chess_model,
            fields=[
                position,
                f"<p>{question_move_string}</p>", 
                f"<img src=\"{question_image_path}.svg\">", 
                f"<p>{question_comment_string}</p>", 
                f"<p>{answer_move_string}</p>", 
                f"<img src=\"{answer_image_path}.svg\">", 
                f"<p>{answer_comment_string}</p>", 
            ]
        )
        chess_deck.add_note(note)

write_cards()

In [ ]:
from os import listdir
from os.path import isfile, join
image_files = ["images/" + f for f in listdir("images/") if isfile(join("images/", f))]

package = genanki.Package(chess_deck)
package.media_files = image_files
package.write_to_file('output/Caro Kann Defense.apkg')